In [1]:
import time
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import missingno as msno
import plotly.graph_objects as go
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import f_classif, mutual_info_classif, VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
np.random.seed(42)


#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Allow long lines

In [2]:
# Specify the path to the CSV file in the parent directory
file_path = "../Dataset/pd_speech_features/pd_speech_features.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, sep=',',skiprows=1)

In [3]:
df.head()

id  gender      PPE      DFA     RPDE  numPulses  numPeriodsPulses  \
0   0       1  0.85247  0.71826  0.57227        240               239   
1   0       1  0.76686  0.69481  0.53966        234               233   
2   0       1  0.85083  0.67604  0.58982        232               231   
3   1       0  0.41121  0.79672  0.59257        178               177   
4   1       0  0.32790  0.79782  0.53028        236               235   

   meanPeriodPulses  stdDevPeriodPulses  locPctJitter  locAbsJitter  \
0          0.008064            0.000087       0.00218      0.000018   
1          0.008258            0.000073       0.00195      0.000016   
2          0.008340            0.000060       0.00176      0.000015   
3          0.010858            0.000183       0.00419      0.000046   
4          0.008162            0.002669       0.00535      0.000044   

   rapJitter  ppq5Jitter  ddpJitter  locShimmer  locDbShimmer  apq3Shimmer  \
0    0.00067     0.00129    0.00200     0.05883         0.517      0.03011   
1    0.00052     0.00112    0.00157     0.05516         0.502      0.02320   
2    0.00057     0.00111    0.00171     0.09902         0.897      0.05094   
3    0.00149     0.00268    0.00446     0.05451         0.527      0.02395   
4    0.00166     0.00227    0.00499     0.05610         0.497      0.02909   

   apq5Shimmer  apq11Shimmer  ddaShimmer  meanAutoCorrHarmonicity  \
0      0.03496       0.04828     0.09034                 0.970805   
1      0.03675       0.06195     0.06961                 0.984322   
2      0.06497       0.07772     0.15282                 0.974846   
3      0.02857       0.04462     0.07185                 0.968343   
4      0.03327       0.05278     0.08728                 0.975754   

   meanNoiseToHarmHarmonicity  meanHarmToNoiseHarmonicity  minIntensity  \
0                    0.036223                      18.995     69.997496   
1                    0.017974                      21.497     67.415903   
2                    0.026313                      17.651     62.661706   
3                    0.042003                      19.865     76.306989   
4                    0.027139                      19.557     76.645686   

   maxIntensity  meanIntensity          f1           f2           f3  \
0     76.088046      72.465512  539.342735  1031.849040  2447.162183   
1     73.046374      71.528945  564.363615  1016.367294  2383.565201   
2     71.633549      68.086583  548.444604  1032.406341  2357.826954   
3     81.000749      79.190593  819.529588  1201.813897  3154.035654   
4     80.937258      79.183495  846.796144  1215.346469  3201.513132   

            f4          b1          b2          b3          b4  GQ_prc5_95  \
0  3655.054806  101.092218   83.147440  255.214830  396.643631     0.77778   
1  3498.681572   58.465428   86.487292  248.357127  218.229722     0.81250   
2  3678.128717  160.387771   54.685168  151.694847   84.240339     0.81818   
3  4122.163933  238.667052  191.984916  573.752909  526.147599     0.98548   
4  4085.456839  402.216738  210.061394  203.637106  384.611697     0.97847   

   GQ_std_cycle_open  GQ_std_cycle_closed  GNE_mean  GNE_std  GNE_SNR_TKEO  \
0            11.7245               2.8277   1.17300  0.26512      0.083127   
1            13.8284               2.8908   1.02210  0.22004      0.127410   
2            26.9273               2.6975   0.84951  0.15756      0.116890   
3           139.5744               1.6961   0.83405  0.17295      0.147370   
4           102.0549              15.4045   0.83556  0.16210      0.151990   

   GNE_SNR_SEO  GNE_NSR_TKEO  GNE_NSR_SEO  VFER_mean  VFER_std  VFER_entropy  \
0  1200445.612        1.5347       3.0152   0.000463  0.000260       0.83621   
1  1298455.445        1.6029       3.0600   0.000615  0.000295       1.04550   
2  1272869.841        1.6223       3.0309   0.000360  0.000266       0.63668   
3  1932289.206        1.6717       3.0293   0.000196  0.000194       0.45452   
4  1861807.802        1.6781   

# Filtrar o Dataset para apenas apresentar dados do Sexo Feminino e sexo Masculino

In [4]:
female_df = df[df['gender'] == 0]
male_df = df[df['gender'] == 1]
female_df.shape
male_df.shape

(390, 755)

# Drop da Coluna de Id

In [5]:
female_df = female_df.drop("id", axis=1)
male_df = male_df.drop("id", axis=1)

# Normalização dos dois Datasets (Feminino e Masculino)

In [6]:
features = female_df.columns

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Scale the features
df_female_scaled = pd.DataFrame(scaler.fit_transform(female_df[features]), columns=features)

In [7]:
features = male_df.columns

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Scale the features
df_male_scaled = pd.DataFrame(scaler.fit_transform(male_df[features]), columns=features)

# Correlation

In [8]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything else
# without any further insight.

def correlation(dataset, threshold):
    
    # create a set where I will store the names of correlated columns
    col_corr = set()
    
    # create the correlation matrix
    corr_matrix = dataset.corr()
    
    # for each feature in the dataset (columns of the correlation matrix)
    for i in range(len(corr_matrix.columns)):
        
        # check with other features
        for j in range(i):
            
            # if the correlation is higher than a certain threshold
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                
                # print correlation, and variables examined
                # keep in mind that the columns and rows of the dataframe are identical
                # so we can identify the features being examned by looking for i,j
                # in the column names
                print(abs(corr_matrix.iloc[i, j]), corr_matrix.columns[i], corr_matrix.columns[j])
                
                # get the name of the correlated feature
                colname = corr_matrix.columns[j]
                
                # and add it to our correlated set
                col_corr.add(colname)
                
    return col_corr

In [9]:
corr_features_female = correlation(df_female_scaled, 0.8)
corr_features_male = correlation(df_male_scaled, 0.8)
print(len(set(corr_features_female)))
print(len(set(corr_features_male)))

0.9999377154191265 numPeriodsPulses numPulses
0.8944194152814179 meanPeriodPulses numPulses
0.8942202324314605 meanPeriodPulses numPeriodsPulses
0.9567617276889551 locAbsJitter locPctJitter
0.9576124197306861 rapJitter locPctJitter
0.9637890493373078 rapJitter locAbsJitter
0.9655597415588171 ppq5Jitter locPctJitter
0.9527005799849387 ppq5Jitter locAbsJitter
0.9690081594863994 ppq5Jitter rapJitter
0.9576367320794035 ddpJitter locPctJitter
0.9638538905669238 ddpJitter locAbsJitter
0.9999949562401894 ddpJitter rapJitter
0.969061236106539 ddpJitter ppq5Jitter
0.9954074546347035 locDbShimmer locShimmer
0.9817976193709238 apq3Shimmer locShimmer
0.9748964285479385 apq3Shimmer locDbShimmer
0.9827917101819044 apq5Shimmer locShimmer
0.975185188709493 apq5Shimmer locDbShimmer
0.966774730630555 apq5Shimmer apq3Shimmer
0.9063252890337412 apq11Shimmer locShimmer
0.9067921314572864 apq11Shimmer locDbShimmer
0.8559731001399625 apq11Shimmer apq3Shimmer
0.9225359286789021 apq11Shimmer apq5Shimmer
0.9817

0.8495858915634806 det_TKEO_mean_9_coef Ea
0.8125206145255657 det_TKEO_mean_9_coef Ed_8_coef
0.9044609660972914 det_TKEO_mean_9_coef Ed_10_coef
0.9864574628863576 det_TKEO_mean_9_coef det_entropy_shannon_9_coef
0.86310760197848 det_TKEO_mean_10_coef Ed_10_coef
0.8788773863024048 det_TKEO_mean_10_coef det_entropy_shannon_8_coef
0.9991019862185603 det_TKEO_mean_10_coef det_entropy_shannon_10_coef
0.83119841806263 det_TKEO_mean_10_coef det_TKEO_mean_8_coef
0.9594337145410525 det_TKEO_std_1_coef det_entropy_shannon_1_coef
0.9209926609320055 det_TKEO_std_1_coef det_entropy_shannon_2_coef
0.9507041691157739 det_TKEO_std_1_coef det_TKEO_mean_1_coef
0.9041288126613526 det_TKEO_std_1_coef det_TKEO_mean_2_coef
0.8026202232073038 det_TKEO_std_2_coef Ed_4_coef
0.9384191488500085 det_TKEO_std_2_coef det_entropy_shannon_1_coef
0.9676622556403783 det_TKEO_std_2_coef det_entropy_shannon_2_coef
0.8041019552334449 det_TKEO_std_2_coef det_entropy_shannon_3_coef
0.8002596602205657 det_TKEO_std_2_coef det_

0.814777999168545 det_LT_TKEO_mean_5_coef Ed_5_coef
0.8112219376468335 det_LT_TKEO_mean_5_coef det_entropy_shannon_3_coef
0.981117223427003 det_LT_TKEO_mean_5_coef det_entropy_shannon_5_coef
0.9771308924928841 det_LT_TKEO_mean_5_coef det_TKEO_mean_5_coef
0.9742557663824617 det_LT_TKEO_mean_5_coef det_TKEO_std_5_coef
0.9453646470011623 det_LT_TKEO_mean_5_coef Ed2_5_coef
0.8703169902599405 det_LT_TKEO_mean_5_coef det_LT_entropy_shannon_5_coef
0.8020323571442458 det_LT_TKEO_mean_5_coef det_LT_TKEO_mean_4_coef
0.9274495680039689 det_LT_TKEO_mean_6_coef det_entropy_shannon_6_coef
0.9359715164924967 det_LT_TKEO_mean_6_coef det_TKEO_mean_6_coef
0.9280452626238097 det_LT_TKEO_mean_6_coef det_TKEO_std_6_coef
0.9807295796800674 det_LT_TKEO_mean_6_coef Ed2_6_coef
0.8320870806188124 det_LT_TKEO_mean_6_coef det_LT_entropy_shannon_6_coef
0.9345996033898123 det_LT_TKEO_mean_7_coef det_entropy_shannon_7_coef
0.9681387283372778 det_LT_TKEO_mean_7_coef det_TKEO_mean_7_coef
0.9472561682477508 det_LT_TKEO

0.9354916249844176 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_1_coef
0.9436054048098371 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_2_coef
0.9538157767671301 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_3_coef
0.9617564384111243 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_4_coef
0.9638629466781788 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_5_coef
0.9629910495084032 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_6_coef
0.9614008564353443 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_7_coef
0.9594262706437968 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_8_coef
0.9593792640121939 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_9_coef
0.9594855046583748 app_LT_TKEO_std_6_coef app_LT_entropy_shannon_10_coef
0.8403541770749532 app_LT_TKEO_std_6_coef app_LT_entropy_log_5_coef
0.9081861790508049 app_LT_TKEO_std_6_coef app_LT_entropy_log_6_coef
0.892795184031062 app_LT_TKEO_std_6_coef app_LT_entropy_log_7_coef
0.9452850606885207 app_LT_TKEO_std_6_coef app_LT_entropy_log_8_coef
0.945298

0.8870982996949954 tqwt_entropy_shannon_dec_5 tqwt_entropy_shannon_dec_4
0.8142481128855206 tqwt_entropy_shannon_dec_6 tqwt_entropy_shannon_dec_5
0.8635580803025598 tqwt_entropy_shannon_dec_8 tqwt_energy_dec_8
0.9045816206712027 tqwt_entropy_shannon_dec_8 tqwt_entropy_shannon_dec_7
0.8770808292068014 tqwt_entropy_shannon_dec_9 tqwt_entropy_shannon_dec_8
0.8258593029022278 tqwt_entropy_shannon_dec_10 tqwt_energy_dec_10
0.8392180166991128 tqwt_entropy_shannon_dec_10 tqwt_entropy_shannon_dec_9
0.8140831700125893 tqwt_entropy_shannon_dec_11 tqwt_entropy_shannon_dec_10
0.804219116963004 tqwt_entropy_shannon_dec_12 tqwt_energy_dec_12
0.8335013416174815 tqwt_entropy_shannon_dec_12 tqwt_energy_dec_13
0.8840911921727849 tqwt_entropy_shannon_dec_12 tqwt_entropy_shannon_dec_11
0.8752657890535745 tqwt_entropy_shannon_dec_13 tqwt_energy_dec_13
0.914729373540343 tqwt_entropy_shannon_dec_13 tqwt_entropy_shannon_dec_12
0.8553624504834798 tqwt_entropy_shannon_dec_14 tqwt_energy_dec_14
0.882919996105343

...

0.8073974852133857 tqwt_TKEO_std_dec_33 tqwt_TKEO_std_dec_32
0.8116721731394874 tqwt_TKEO_std_dec_34 tqwt_entropy_shannon_dec_34
0.850015910595596 tqwt_TKEO_std_dec_34 tqwt_entropy_shannon_dec_35
0.9097027271881276 tqwt_TKEO_std_dec_34 tqwt_TKEO_mean_dec_34
0.873434909598701 tqwt_TKEO_std_dec_34 tqwt_TKEO_mean_dec_35
0.9195987037461821 tqwt_TKEO_std_dec_34 tqwt_TKEO_std_dec_33
0.9310475666089052 tqwt_TKEO_std_dec_35 tqwt_TKEO_mean_dec_35
0.8331431147641402 tqwt_TKEO_std_dec_35 tqwt_TKEO_mean_dec_36
0.800974465489844 tqwt_TKEO_std_dec_35 tqwt_TKEO_std_dec_34
0.9470718494029369 tqwt_TKEO_std_dec_36 tqwt_TKEO_mean_dec_36
0.8056616975629051 tqwt_TKEO_std_dec_36 tqwt_TKEO_std_dec_35
0.8951244148473612 tqwt_meanValue_dec_36 tqwt_medianValue_dec_36
0.9539384645314711 tqwt_stdValue_dec_1 tqwt_entropy_shannon_dec_1
0.859413461053978 tqwt_stdValue_dec_1 tqwt_entropy_shannon_dec_2
0.9186409409247556 tqwt_stdValue_dec_1 tqwt_TKEO_mean_dec_1
0.8405271509898947 tqwt_stdValue_dec_1 tqwt_TKEO_mean_dec

0.9075938596350194 tqwt_stdValue_dec_28 tqwt_TKEO_mean_dec_28
0.9202183508535844 tqwt_stdValue_dec_29 tqwt_energy_dec_29
0.8608401655414504 tqwt_stdValue_dec_29 tqwt_energy_dec_30
0.8094375837756027 tqwt_stdValue_dec_29 tqwt_entropy_shannon_dec_29
0.9128674803262143 tqwt_stdValue_dec_29 tqwt_TKEO_mean_dec_29
0.8779518958107065 tqwt_stdValue_dec_29 tqwt_TKEO_mean_dec_30
0.9256728405407403 tqwt_stdValue_dec_29 tqwt_TKEO_std_dec_29
0.8215043361344802 tqwt_stdValue_dec_29 tqwt_TKEO_std_dec_30
0.8542120854131325 tqwt_stdValue_dec_30 tqwt_energy_dec_29
0.9282489091761864 tqwt_stdValue_dec_30 tqwt_energy_dec_30
0.8588559330520589 tqwt_stdValue_dec_30 tqwt_entropy_shannon_dec_29
0.9202308243070157 tqwt_stdValue_dec_30 tqwt_TKEO_mean_dec_29
0.9311961912124068 tqwt_stdValue_dec_30 tqwt_TKEO_mean_dec_30
0.9244339299198278 tqwt_stdValue_dec_30 tqwt_TKEO_std_dec_29
0.8704125532761013 tqwt_stdValue_dec_30 tqwt_TKEO_std_dec_30
0.9149225727931298 tqwt_stdValue_dec_30 tqwt_stdValue_dec_29
0.96397007404

0.8431802461698129 tqwt_minValue_dec_19 tqwt_entropy_log_dec_19
0.8220970872520688 tqwt_minValue_dec_19 tqwt_TKEO_mean_dec_19
0.9319095459470194 tqwt_minValue_dec_19 tqwt_TKEO_std_dec_19
0.831386795083632 tqwt_minValue_dec_19 tqwt_stdValue_dec_18
0.9478198311006072 tqwt_minValue_dec_19 tqwt_stdValue_dec_19
0.8442270701526765 tqwt_minValue_dec_19 tqwt_stdValue_dec_20
0.8856440239948179 tqwt_minValue_dec_19 tqwt_minValue_dec_18
0.8598962520434066 tqwt_minValue_dec_20 tqwt_TKEO_mean_dec_20
0.8166779333649555 tqwt_minValue_dec_20 tqwt_TKEO_std_dec_19
0.8731288984246397 tqwt_minValue_dec_20 tqwt_TKEO_std_dec_20
0.805271503776581 tqwt_minValue_dec_20 tqwt_TKEO_std_dec_21
0.9557424130041797 tqwt_minValue_dec_20 tqwt_stdValue_dec_20
0.806388721048253 tqwt_minValue_dec_20 tqwt_stdValue_dec_21
0.8790025449079395 tqwt_minValue_dec_20 tqwt_minValue_dec_19
0.8754498405216036 tqwt_minValue_dec_21 tqwt_TKEO_std_dec_21
0.8499597658532831 tqwt_minValue_dec_21 tqwt_stdValue_dec_20
0.9355713641389108 tqw

0.8870090548219332 tqwt_maxValue_dec_7 tqwt_minValue_dec_6
0.9856970921827761 tqwt_maxValue_dec_7 tqwt_minValue_dec_7
0.8930814516782889 tqwt_maxValue_dec_7 tqwt_minValue_dec_8
0.8053218170817631 tqwt_maxValue_dec_7 tqwt_maxValue_dec_5
0.8903525643904592 tqwt_maxValue_dec_7 tqwt_maxValue_dec_6
0.8165721581704963 tqwt_maxValue_dec_8 tqwt_stdValue_dec_7
0.8557314988810095 tqwt_maxValue_dec_8 tqwt_stdValue_dec_8
0.9004265041755934 tqwt_maxValue_dec_8 tqwt_minValue_dec_7
0.993812967838937 tqwt_maxValue_dec_8 tqwt_minValue_dec_8
0.9040356489716348 tqwt_maxValue_dec_8 tqwt_minValue_dec_9
0.8935002675181402 tqwt_maxValue_dec_8 tqwt_maxValue_dec_7
0.8252089359941079 tqwt_maxValue_dec_9 tqwt_TKEO_std_dec_9
0.8573787126475949 tqwt_maxValue_dec_9 tqwt_stdValue_dec_9
0.8922496729559654 tqwt_maxValue_dec_9 tqwt_minValue_dec_8
0.9888890377538235 tqwt_maxValue_dec_9 tqwt_minValue_dec_9
0.8979193121190774 tqwt_maxValue_dec_9 tqwt_minValue_dec_10
0.8926235236810253 tqwt_maxValue_dec_9 tqwt_maxValue_dec

0.92561321529694 tqwt_skewnessValue_dec_2 tqwt_skewnessValue_dec_1
0.9115860167810156 tqwt_skewnessValue_dec_4 tqwt_skewnessValue_dec_2
0.9130484117635035 tqwt_skewnessValue_dec_25 tqwt_skewnessValue_dec_24
0.917912149119554 tqwt_skewnessValue_dec_26 tqwt_skewnessValue_dec_24
0.9968572727705793 tqwt_skewnessValue_dec_26 tqwt_skewnessValue_dec_25
0.9505891516281268 tqwt_skewnessValue_dec_32 tqwt_skewnessValue_dec_31
0.9034224488302438 tqwt_skewnessValue_dec_33 tqwt_skewnessValue_dec_31
0.9479813706946487 tqwt_skewnessValue_dec_33 tqwt_skewnessValue_dec_32
0.8066625436278049 tqwt_skewnessValue_dec_34 tqwt_skewnessValue_dec_32
0.87354176133049 tqwt_skewnessValue_dec_34 tqwt_skewnessValue_dec_33
0.817855847540224 tqwt_skewnessValue_dec_35 tqwt_skewnessValue_dec_33
0.9627016030470471 tqwt_skewnessValue_dec_35 tqwt_skewnessValue_dec_34
0.9157100224103444 tqwt_skewnessValue_dec_36 tqwt_skewnessValue_dec_34
0.9402961073253373 tqwt_skewnessValue_dec_36 tqwt_skewnessValue_dec_35
0.93970756244000

In [10]:
df_female_scaled.drop(labels=corr_features_female, axis=1, inplace=True)

df_female_scaled.shape

(366, 242)

In [11]:
df_male_scaled.drop(labels=corr_features_male, axis=1, inplace=True)

df_male_scaled.shape

(390, 260)

# Split para dados de treino e teste

In [12]:
X_female=df_female_scaled.drop('class',axis=1)
y_female=df_female_scaled['class']
X_train_female, X_test_female, y_train_female, y_test_female=train_test_split(X_female,y_female,test_size=0.2, random_state=42)

In [13]:
X_male=df_male_scaled.drop('class',axis=1)
y_male=df_male_scaled['class']
X_train_male, X_test_male, y_train_male, y_test_male=train_test_split(X_male,y_male,test_size=0.2, random_state=42)

In [14]:
def evaluate_classifiers(X_train, y_train, X_test, y_test):
    classifiers = {
        'SVM': {
            'model': SVC(),
            'param_grid': {
                'C': [0.1, 1, 10],
                'kernel': ['linear', 'rbf', 'poly'],
                'gamma': ['scale', 'auto']
            }
        },
        'Random Forest': {
            'model': RandomForestClassifier(),
            'param_grid': {
                'n_estimators': [100, 200, 300],
                'criterion': ['gini', 'entropy'],
                'max_depth': [None, 5, 10]
            }
        },
        'KNN': {
            'model': KNeighborsClassifier(),
            'param_grid': {
                'n_neighbors': [3, 5, 7],
                'weights': ['uniform', 'distance'],
                'p': [1, 2]
            }
        },
        'Naive Bayes': {
            'model': GaussianNB(),
            'param_grid': {
                'priors': [None, [0.1, 0.9], [0.3, 0.7], [0.5, 0.5], [0.7, 0.3], [0.9, 0.1]]

            }
        },
        'Decision Tree': {
            'model': DecisionTreeClassifier(),
            'param_grid': {
                'criterion': ['gini', 'entropy'],
                'max_depth': [None, 5, 10]
            }
        },
        'XGBoost': {
            'model': XGBClassifier(),
            'param_grid': {
                'learning_rate': [0.1, 0.01],
                'max_depth': [3, 5, 7],
                'n_estimators': [100, 200, 300]
            }
        },
        'AdaBoost': {
            'model': AdaBoostClassifier(),
            'param_grid': {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.1, 1, 10]
            }
        }
        
    }

    performance_metrics = {}

    for clf_name, clf_info in classifiers.items():
        clf = clf_info['model']
        param_grid = clf_info['param_grid']
        
        start = time.time()
        grid_search = GridSearchCV(clf, param_grid, cv=5)
        grid_search.fit(X_train, y_train)
        best_clf = grid_search.best_estimator_

        y_pred = best_clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred)
        time_execution = time.time() - start
        performance_metrics[clf_name] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc':auc,
            'time_execution':time_execution
        }
        

    return performance_metrics

In [15]:
results_female = evaluate_classifiers(X_train_female, y_train_female, X_test_female, y_test_female)

for clf_name, metrics in results_female.items():
    print(clf_name + " Performance Metrics:")
    print("Accuracy:", metrics['accuracy'])
    print("Precision:", metrics['precision'])
    print("Recall:", metrics['recall'])
    print("F1 Score:", metrics['f1'])
    print("AUC-ROC:", metrics['auc'])
    print("Time:", metrics['time_execution'])
    print()

SVM Performance Metrics:
Accuracy: 0.7972972972972973
Precision: 0.7924528301886793
Recall: 0.9130434782608695
F1 Score: 0.8484848484848485
AUC-ROC: 0.7600931677018634
Time: 0.7973957061767578

Random Forest Performance Metrics:
Accuracy: 0.8648648648648649
Precision: 0.8461538461538461
Recall: 0.9565217391304348
F1 Score: 0.8979591836734695
AUC-ROC: 0.8354037267080746
Time: 29.73150634765625

KNN Performance Metrics:
Accuracy: 0.9054054054054054
Precision: 0.9333333333333333
Recall: 0.9130434782608695
F1 Score: 0.9230769230769231
AUC-ROC: 0.9029503105590063
Time: 0.44206833839416504

Naive Bayes Performance Metrics:
Accuracy: 0.7432432432432432
Precision: 0.8857142857142857
Recall: 0.6739130434782609
F1 Score: 0.7654320987654321
AUC-ROC: 0.765527950310559
Time: 0.1738908290863037

Decision Tree Performance Metrics:
Accuracy: 0.8243243243243243
Precision: 0.8235294117647058
Recall: 0.9130434782608695
F1 Score: 0.865979381443299
AUC-ROC: 0.7958074534161491
Time: 0.6775803565979004

XGBo

In [16]:
results_male = evaluate_classifiers(X_train_male, y_train_male, X_test_male, y_test_male)

for clf_name, metrics in results_male.items():
    print(clf_name + " Performance Metrics:")
    print("Accuracy:", metrics['accuracy'])
    print("Precision:", metrics['precision'])
    print("Recall:", metrics['recall'])
    print("F1 Score:", metrics['f1'])
    print("AUC-ROC:", metrics['auc'])
    print("Time:", metrics['time_execution'])
    print()

SVM Performance Metrics:
Accuracy: 0.8461538461538461
Precision: 0.9137931034482759
Recall: 0.8833333333333333
F1 Score: 0.8983050847457628
AUC-ROC: 0.8027777777777778
Time: 0.8110017776489258

Random Forest Performance Metrics:
Accuracy: 0.8205128205128205
Precision: 0.8108108108108109
Recall: 1.0
F1 Score: 0.8955223880597014
AUC-ROC: 0.6111111111111112
Time: 51.06795120239258

KNN Performance Metrics:
Accuracy: 0.8717948717948718
Precision: 0.9032258064516129
Recall: 0.9333333333333333
F1 Score: 0.9180327868852459
AUC-ROC: 0.8
Time: 0.8762996196746826

Naive Bayes Performance Metrics:
Accuracy: 0.6538461538461539
Precision: 0.8837209302325582
Recall: 0.6333333333333333
F1 Score: 0.7378640776699028
AUC-ROC: 0.6777777777777778
Time: 0.33375024795532227

Decision Tree Performance Metrics:
Accuracy: 0.8461538461538461
Precision: 0.9
Recall: 0.9
F1 Score: 0.9
AUC-ROC: 0.7833333333333334
Time: 1.4423120021820068

XGBoost Performance Metrics:
Accuracy: 0.9230769230769231
Precision: 0.921875